In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import covsirphy as cs
import ruptures as rpt
import tqdm.notebook as tq
import plotly as py
import plotly.graph_objs as go
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

In [ ]:
data_loader = cs.DataLoader(directory="kaggle/input")
jhu_data = data_loader.jhu()
population_data = data_loader.population()
oxcgrt_data = data_loader.oxcgrt()

measures_delay = pd.DataFrame(columns=["Country", "Measures_Delay"])
measures_delay.set_index("Country", inplace=True)

plt.rcParams['figure.figsize'] = 4, 3


countries = ["Italy", "Japan", "China", "India", "USA",
                          "Greece", "Brazil", "Russia", "Spain", "France",
                          "United Kingdom", "Germany", "Netherlands", "Czech Republic",
                          "Poland", "Sweden", "Austria", "Australia",
                          "Switzerland", "Portugal", "South Africa"]

countries = jhu_data.cleaned()["Country"].unique()

In [ ]:
measures_delay = pd.DataFrame(columns=["Country", "Measures_Delay"])
measures_delay.set_index("Country", inplace=True)

for i in tq.tqdm(range(len(countries))):
    country = countries[i]
    
    try:
        scn = cs.Scenario(jhu_data, population_data, country)
    except:
        print(country)
    records = scn.records(show_figure=False,
                                        variables=["Confirmed", "Infected",
                                                  "Fatal", "Recovered"])
    records["Confirmed_diff"] = records["Confirmed"].diff()
    records.replace(0, np.nan, inplace=True)
    records.fillna(method="bfill", inplace=True)
    records.fillna(method="ffill", inplace=True)

    oxcgrt_df = oxcgrt_data.subset(country=country).set_index("Date")
    records = records.set_index("Date")
    records["Stringency_index"] = oxcgrt_df["Stringency_index"]
    
    df = records.pivot_table(index="Confirmed", values="Stringency_index")
    df_run = df.copy()

    # Convert index to serial numbers
    serial_df = pd.DataFrame(np.arange(1, df_run.index.max() + 1, 1))
    serial_df.index += 1
    df_run = df_run.join(serial_df, how="outer")
    series = df_run.reset_index(drop=True).iloc[:, 0]
    series = series.dropna()

    records = records.reset_index()
    
    
    # Convert index values to Susceptible values
    reset_series = series.reset_index(drop=True)
    reset_series.index += 1
    reset_series
    
    # Detection with Ruptures
    algorithm = rpt.Pelt(model="rbf", jump=1, min_size=7)
    
    try:
        results = algorithm.fit_predict(series.values, pen=0.5)
    except: 
        continue

    new_confirmed_df = reset_series[results].reset_index()
    new_confirmed_df = new_confirmed_df.interpolate(method="linear").dropna().astype(np.float64)
    
    # Convert new_confirmed values to dates
    df = pd.merge_asof(
        new_confirmed_df.sort_values("Stringency_index"),
        df.reset_index().sort_values("Stringency_index"),
        on="Stringency_index", direction="nearest"
    )
    
    df["periods"] = df["index"].sort_values(ignore_index=True)
    df["periods"] = df["periods"].diff()
    
    # Filter out outliers (very long periods)
#     df_filtered = df[(df["periods"] < df["periods"].quantile(0.90))]
    df_filtered = df[(df["periods"] < 35)]

    measures_delay.loc[country] = df_filtered["periods"].mean()

print(measures_delay)
print(measures_delay.mean())
measures_delay = measures_delay.reset_index()

In [ ]:
i = 9
data = dict (
    type = 'choropleth',
    locations = measures_delay["Country"].values,
    locationmode='country names',
    z=measures_delay["Measures_Delay"].values,
    colorscale=px.colors.named_colorscales()[i],
    colorbar=dict(thickness=30,
                           ticklen=7,
                           tickfont=dict(size=20)),
)

map = go.Figure(data=[data], 
                        layout = go.Layout(
                        title="Average number of days before effect of interventions is detected",
                        geo=dict(bgcolor= 'rgba(233,233,233,0.3)')))

py.offline.plot(map)

In [ ]:
named_colorscales = px.colors.named_colorscales()
named_colorscales

In [ ]:
list(set(countries) - set(measures_delay["Country"]))

In [ ]:
import covsirphy as cs

data_loader = cs.DataLoader(directory="kaggle/input")
jhu_data = data_loader.jhu()
population_data = data_loader.population()
oxcgrt_data = data_loader.oxcgrt()


In [ ]:
country = "Netherlands"

scn = cs.Scenario(jhu_data, population_data, country)